In [170]:
f1 = open('user_5k_avg', 'r').read()
user_avg = eval(f1)

f2 = open('business_5k_avg', 'r').read()
business_avg = eval(f2)

f1 = open('review_5k_user', 'r').read()
review_5k_user = eval(f1)

f2 = open('review_5k_business', 'r').read()
review_5k_business = eval(f2)

f3 = open('review_5k_rating', 'r').read()
review_5k_rating = eval(f3)

f4 = open('relation_5k', 'r').read()
relation_5k = eval(f4)


In [190]:
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.utils.extmath import randomized_svd
from sklearn.metrics import mean_squared_error
import networkx as nx

In [142]:
num_user = 4929
num_business = 2686
num_review = 49486

In [143]:
row = np.array([])
col = np.array([])
val = np.array([])

In [144]:
Tij = csr_matrix((val,(row,col)), shape=(num_user,num_user)).toarray()
for u in relation_5k:
    for f in u[1]:
        Tij[u[0]][f] = 1

In [145]:
Rij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_review):
    Rij[review_5k_user[r]][review_5k_business[r]] = review_5k_rating[r]

In [146]:
Sij_norm = []
for u in xrange(num_user):
    Sij_norm.append(math.sqrt(np.dot(Rij[u], Rij[u])))

In [147]:
Sij = np.dot(Rij, Rij.T)

In [148]:
for i in xrange(num_user):
    for j in xrange(num_user):
        if Sij_norm[i] and Sij_norm[j]:
            Sij[i][j] = Sij[i][j]/Sij_norm[i]/Sij_norm[j]
        else: 
            Sij[i][j] = 0

In [154]:
import networkx as nx

node_list = []
for u in xrange(num_user):
    node_list.append(u)


In [155]:
G=nx.Graph()
G.add_nodes_from(node_list)
for u in relation_5k:
    for f in u[1]:
        G.add_edge(u[0], f)

In [156]:
PR = nx.pagerank(G)

In [157]:
from operator import itemgetter
sorted_PR = sorted(PR.items(), key=itemgetter(1), reverse=True)

rank = {}
for u in xrange(num_user):
    rank[sorted_PR[u][0]] = u

Ri = []
for u in xrange(num_user):
    Ri.append(rank[u]+1)

Wi = []
for ri in Ri:
    Wi.append(1.0/(1.0+ math.log(ri)))    

Wij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        if Rij[i][j] > 0:
            Wij[i][j] = math.sqrt(Wi[i])

In [199]:
uv = []
for r in xrange(len(review_5k_rating)):
    ubb = user_avg[review_5k_user[r]] + business_avg[review_5k_business[r]] - mu
    delta = review_5k_rating[r] - ubb
    weight = Wij[review_5k_user[r]][review_5k_business[r]]
    uv.append(weight*delta) 

row = np.array(review_5k_business)
col = np.array(review_5k_user)
val = np.array(uv)

ori = csr_matrix((val,(row,col)), shape=(len(business_avg),len(user_avg))).toarray()

In [200]:
n_comp = 30 # k
n_iter = 15
U, S, VT = randomized_svd(ori, n_components=n_comp, n_iter=n_iter, random_state=None)


In [208]:
len(S[0])

30

In [201]:
sum(ori[0])

-1.0527376479251418

In [210]:
now = np.dot(np.dot(U,S),VT)

rmse = mean_squared_error(ori, now)

In [211]:
rmse

0.00032205946942083551